# Data Wrangling with Spark SQL Quiz

This quiz uses the same dataset and most of the same questions from the earlier "Quiz - Data Wrangling with Data Frames Jupyter Notebook." For this quiz, however, use Spark SQL instead of Spark Data Frames.

In [1]:
from pyspark.sql import SparkSession

# TODOS: 
# 1) import any other libraries you might need
# 2) instantiate a Spark session 
# 3) read in the data set located at the path "data/sparkify_log_small.json"
# 4) create a view to use with your SQL queries
# 5) write code to answer the quiz questions 

spark = SparkSession \
    .builder \
    .appName("Spark SQL Quiz") \
    .getOrCreate()

user_log = spark.read.json("file:/home/workspace/data/sparkify_log_small.json")

In [2]:
user_log.printSchema()

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: long (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)



- Since I am gonna be using Spark SQL, it is necessary to create a temp view `log_table`

In [3]:
user_log.createOrReplaceTempView("log_table")

# Question 1

Which page did user id ""(empty string) NOT visit?

In [4]:
spark.sql('''
SELECT DISTINCT 
    page
FROM log_table
EXCEPT
SELECT DISTINCT
    page
FROM log_table
WHERE UserId = ''
''').show()

+----------------+
|            page|
+----------------+
|Submit Downgrade|
|       Downgrade|
|          Logout|
|   Save Settings|
|        Settings|
|        NextSong|
|         Upgrade|
|           Error|
|  Submit Upgrade|
+----------------+



# Question 2 - Reflect

Why might you prefer to use SQL over data frames? Why might you prefer data frames over SQL?

# Question 3

How many female users do we have in the data set?

In [5]:
spark.sql('''
SELECT 
    COUNT(DISTINCT userId) AS Number_of_female_users
FROM log_table
WHERE gender = 'F'
''').show()

+----------------------+
|Number_of_female_users|
+----------------------+
|                   462|
+----------------------+



# Question 4

How many songs were played from the most played artist?

In [6]:
spark.sql('''
SELECT artist, COUNT(song) AS Number_of_songs
FROM log_table
GROUP BY artist
ORDER BY COUNT(song) DESC
''').show(1)

+--------+---------------+
|  artist|Number_of_songs|
+--------+---------------+
|Coldplay|             83|
+--------+---------------+
only showing top 1 row



# Question 5 (challenge)

How many songs do users listen to on average between visiting our home page? Please round your answer to the closest integer.

In [19]:
# Create is_home variable 1 for home and 0 for NextSong
is_home = spark.sql('''
                    SELECT 
                        userId, page, ts,
                        CASE 
                            WHEN page = 'Home' THEN 1
                            ELSE 0
                        END AS is_home
                    FROM log_table
                    WHERE (page = 'Home') or (page = 'NextSong')
''')

# Lets put the variable in a view to be accessed later
is_home.createOrReplaceTempView("is_home_table")

cummulative_song = spark.sql('''
                            SELECT 
                                *, SUM(is_home) OVER(PARTITION BY userId
                                                        ORDER BY ts DESC
                                                        ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) AS period
                            FROM is_home_table
''')

# Lets put cummulative_song in a view
cummulative_song.createOrReplaceTempView('period_table')

spark.sql('''
    WITH CTE_count AS
    (
        SELECT COUNT(*) AS count_result
        FROM period_table
        WHERE page = 'NextSong'
        GROUP BY userId, period
    )
    SELECT ROUND(AVG(count_result), 2) AS average_visits_by_user
    FROM CTE_count
''').show()

+----------------------+
|average_visits_by_user|
+----------------------+
|                   6.9|
+----------------------+

